In [4]:
import Pkg; Pkg.add("QuantumCumulants")


   Resolving package versions...
      Compat entries added for 
     Project No packages added to or removed from `C:\Users\Vedant\Downloads\Projects\Jaynes Cummings Hubbard Model Simulation\Project.toml`
    Manifest No packages added to or removed from `C:\Users\Vedant\Downloads\Projects\Jaynes Cummings Hubbard Model Simulation\Manifest.toml`


In [ ]:
import Pkg; Pkg.add("ModelingToolkit")


   Resolving package versions...
      Compat entries added for ModelingToolkit
    Updating `C:\Users\Vedant\Downloads\Projects\Jaynes Cummings Hubbard Model Simulation\Project.toml`
⌅ [961ee093] + ModelingToolkit v10.32.1
    Manifest No packages added to or removed from `C:\Users\Vedant\Downloads\Projects\Jaynes Cummings Hubbard Model Simulation\Manifest.toml`


In [ ]:
import Pkg; Pkg.add("OrdinaryDiffEq")

   Resolving package versions...
   Installed OrdinaryDiffEqRKN ─────────────────── v1.5.0
   Installed OrdinaryDiffEqPRK ─────────────────── v1.4.0
   Installed OrdinaryDiffEqFIRK ────────────────── v1.16.0
   Installed OrdinaryDiffEqRosenbrock ──────────── v1.18.1
   Installed OrdinaryDiffEqSDIRK ───────────────── v1.7.0
   Installed OrdinaryDiffEqAdamsBashforthMoulton ─ v1.5.0
   Installed OrdinaryDiffEqSSPRK ───────────────── v1.7.0
   Installed OrdinaryDiffEqStabilizedRK ────────── v1.4.0
   Installed NLSolversBase ─────────────────────── v8.0.0
   Installed OrdinaryDiffEqExplicitRK ──────────── v1.4.0
   Installed OrdinaryDiffEqLinear ──────────────── v1.6.0
   Installed OrdinaryDiffEqPDIRK ───────────────── v1.6.0
   Installed OrdinaryDiffEqQPRK ────────────────── v1.4.0
   Installed OrdinaryDiffEqFunctionMap ─────────── v1.5.0
   Installed GenericSchur ──────────────────────── v0.5.6
   Installed OrdinaryDiffEqExponentialRK ───────── v1.8.0
   Installed OrdinaryDiffEqStabilized

In [ ]:
using QuantumCumulants
using OrdinaryDiffEq
using ModelingToolkit

# --- 1. Define Lattice Parameters ---
Lx, Ly = 2, 2   # Change to 5, 5 later
N = Lx * Ly
J = 0.1         # Hopping
g = 0.05        # Coupling
kappa = 1.0     # Decay rate

# --- 2. Define the "Unit Cell" (One Site) ---
# We create ONE Hilbert space definition for a generic site
h_cav = FockSpace(:cavity)
h_atom = NLevelSpace(:atom, 2)
h_site = h_cav ⊗ h_atom # The space of one site

# Define operators for this generic site
# a_unit: acts on component 1 (cavity)
a_unit = Destroy(h_site, :a, 1) 
# σ_unit: acts on component 2 (atom), transition g(1)->e(2)
σ_unit = Transition(h_site, :σ, 2, 1, 2) 

# --- 3. Create Lattice Operators (Indexed) ---
# This creates a wrapper so a(i) is the operator on site i
a(i) = IndexedOperator(a_unit, i)
σ(i) = IndexedOperator(σ_unit, i)
σz(i) = IndexedOperator(Transition(h_site, :σz, 2, 2, 2) - Transition(h_site, :σz, 1, 1, 2), i)

# --- 4. Construct Hamiltonian ---
# Now we can just sum over indices, and the library knows a(i) and a(j) are compatible
H = 0
for i in 1:N
    # Local Jaynes-Cummings
    # Note: We must use the indexed operators a(i) and σ(i)
    H += 1.0 * a(i)' * a(i) + 1.0 * σz(i) + g * (a(i)' * σ(i) + a(i) * σ(i)')
    
    # Hopping (Nearest Neighbors)
    x = (i-1) % Lx
    y = div(i-1, Lx)
    
    # Hop Right
    if x < Lx-1
        j = i + 1
        H += -J * (a(i)' * a(j) + a(i) * a(j)')
    end
    # Hop Up
    if y < Ly-1
        j = i + Lx
        H += -J * (a(i)' * a(j) + a(i) * a(j)')
    end
end

# --- 5. Derive Mean Field Equations ---
# We want to track <a_i> and <σ_i> for all sites
ops_list = [a(i) for i in 1:N]
append!(ops_list, [σ(i) for i in 1:N])

println("Deriving equations... (this may take a moment)")
eqs = meanfield(ops_list, H; order=1) 

# --- 6. Solve ---
# Initial state: Put 1 photon in site 1 (real part=1.0)
u0 = zeros(ComplexF64, length(eqs))
u0[1] = 1.0 + 0.0im # Set <a_1> = 1

@named sys = ODESystem(eqs)
prob = ODEProblem(sys, u0, (0.0, 10.0))
sol = solve(prob, Tsit5())

# --- 7. Plotting ---
using Plots
# Plot photon number expectation |<a>|^2 approx for first few sites
plot(sol.t, abs2.(sol[a(1)]), label="Site 1 (Source)")
plot!(sol.t, abs2.(sol[a(2)]), label="Site 2")

Deriving equations... (this may take a moment)


LoadError: StackOverflowError: